In [ ]:
from IPython.display import display, Javascript, HTML
import ipywidgets as ipw
from src import widgets, utils

In [ ]:
CONFIG_FILENAME = "config/config.json"
CONFIG = utils.read_json(CONFIG_FILENAME)
OPENBIS_OBJECT_TYPES = utils.read_json("metadata/object_types.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis_aiida(
    "https://openbis-empa-dev205.ethz.ch/"
)

In [ ]:
if OPENBIS_SESSION:
    increase_buttons_size = HTML(data="".join(CONFIG["save_home_buttons_settings"]))
    results_vbox = widgets.CreateResultsWidget(OPENBIS_SESSION)

home_button = ipw.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Main menu",
    icon="home",
    layout=ipw.Layout(width="100px", height="50px"),
)

save_button = ipw.Button(
    description="",
    disabled=False,
    button_style="",
    tooltip="Save",
    icon="save",
    layout=ipw.Layout(width="100px", height="50px"),
)

save_close_buttons_hbox = ipw.HBox([save_button, home_button])

In [ ]:
def close_notebook(b):
    display(Javascript(data='window.location.replace("home.ipynb")'))


def save_results(b):
    project_id = results_vbox.select_project_widget.project_dropdown.value
    if project_id == "-1":
        return
    else:
        project_collections = utils.get_openbis_collections(
            OPENBIS_SESSION, project=project_id
        )
        results_collection = None
        for collection in project_collections:
            if collection.props["name"] == "Results":
                results_collection = collection

        if results_collection is None:
            results_collection = utils.create_openbis_collection(
                OPENBIS_SESSION,
                type="COLLECTION",
                project=project_id,
                code="RESULTS_COLLECTION",
                props={"name": "Results"},
            )

        simulations_ids = list(results_vbox.select_simulations_selector.value)
        measurements_ids = list(results_vbox.select_measurements_selector.value)
        analysis_ids = list(results_vbox.select_analysis_selector.value)

        results_parents = simulations_ids + measurements_ids + analysis_ids

        results_type = OPENBIS_OBJECT_TYPES["Result"]

        results_properties = {
            "name": results_vbox.name_textbox.value,
            "description": results_vbox.description_textbox.value,
            "comments": results_vbox.comments_textbox.value,
        }

        results_object = utils.create_openbis_object(
            OPENBIS_SESSION,
            type=results_type,
            parents=results_parents,
            collection=results_collection,
            props=results_properties,
        )

        utils.upload_datasets(
            OPENBIS_SESSION,
            results_object,
            results_vbox.support_files_uploader,
            "ATTACHMENT",
        )

        # Reset widgets
        results_vbox.select_project_widget.project_dropdown.value = "-1"

# Create results record in openBIS

In [ ]:
if OPENBIS_SESSION:
    display(results_vbox)
    display(increase_buttons_size)

home_button.on_click(close_notebook)
save_button.on_click(save_results)
display(save_close_buttons_hbox)